In [31]:
import pandas as pd

# read scraped output
df = pd.read_csv("output.csv", names=["url","artist_country","artist","location","date_string","insta","spotify"])

# read ratings and comments from google sheet
SHEET_ID = '1uu3_S_e2zR5O2cTbknIUh4bG1GymBtdwieFwZynTPRY'
SHEET_NAME = 'transform'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df_ratings = pd.read_csv(url,usecols=['🧑‍🎤 artist 👩‍🎤','⭐ Mikael rating','⭐ Line rating','💭 comment'])

# rename
df_ratings = df_ratings.rename(columns={'🧑‍🎤 artist 👩‍🎤':'artist','⭐ Mikael rating':'m_rating','⭐ Line rating':'l_rating','💭 comment':'comment'})

In [32]:
df = df.join(df_ratings, how='outer', lsuffix='_left', rsuffix='_right')
df.head()


,url,artist_country,artist_left,location,date_string,insta,spotify,artist_right,m_rating,l_rating,comment
0,https://www.roskilde-festival.dk/da/years/2023...,US,Kendrick Lamar,Orange,Onsdag 28. juni 2023,https://www.instagram.com/kendricklamar/,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,Kendrick Lamar,🤯JA!,NaN,NaN
1,https://www.roskilde-festival.dk/da/years/2023...,US,Queens Of The Stone Age,Arena,Onsdag 28. juni 2023,https://www.instagram.com/queensofthestoneage,https://open.spotify.com/artist/4pejUc4iciQfgd...,Queens Of The Stone Age,😌Tømmermændsmusik,NaN,NaN
2,https://www.roskilde-festival.dk/da/years/2023...,DK,Blæst,Orange,Onsdag 28. juni 2023,https://www.instagram.com/blaestband/,https://open.spotify.com/artist/3elCsuJ1JjDeAd...,Fever Ray,😌Tømmermændsmusik,NaN,NaN
3,https://www.roskilde-festival.dk/da/years/2023...,SE,Fever Ray,Arena,Onsdag 28. juni 2023,https://www.instagram.com/feverray/,https://open.spotify.com/artist/5hE6NCoobhyEu6...,Rema,👍Ja,NaN,commercial african pop. So smooth and well-pro...
4,https://www.roskilde-festival.dk/da/years/2023...,US,Lorna Shore,Avalon,Onsdag 28. juni 2023,https://www.instagram.com/lornashore/?hl=da,https://open.spotify.com/artist/6vXYoy8ouRVib3...,Armand Hammer,😌Tømmermændsmusik,NaN,chill jazz akkorder + rap


In [33]:
# Extract day, month, and year into separate columns
df[['day', 'month', 'year']] = df['date_string'].str.extract(r'(\d{1,2})\.\s*(\w+)\s*(\d{4})')
df['start_time'] = df['date_string'].str.extract(r'(\d{2}:\d{2})')

df.head()

,url,artist_country,artist_left,location,date_string,insta,spotify,artist_right,m_rating,l_rating,comment,day,month,year,start_time
0,https://www.roskilde-festival.dk/da/years/2023...,US,Kendrick Lamar,Orange,Onsdag 28. juni 2023,https://www.instagram.com/kendricklamar/,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,Kendrick Lamar,🤯JA!,NaN,NaN,28,juni,2023,NaN
1,https://www.roskilde-festival.dk/da/years/2023...,US,Queens Of The Stone Age,Arena,Onsdag 28. juni 2023,https://www.instagram.com/queensofthestoneage,https://open.spotify.com/artist/4pejUc4iciQfgd...,Queens Of The Stone Age,😌Tømmermændsmusik,NaN,NaN,28,juni,2023,NaN
2,https://www.roskilde-festival.dk/da/years/2023...,DK,Blæst,Orange,Onsdag 28. juni 2023,https://www.instagram.com/blaestband/,https://open.spotify.com/artist/3elCsuJ1JjDeAd...,Fever Ray,😌Tømmermændsmusik,NaN,NaN,28,juni,2023,NaN
3,https://www.roskilde-festival.dk/da/years/2023...,SE,Fever Ray,Arena,Onsdag 28. juni 2023,https://www.instagram.com/feverray/,https://open.spotify.com/artist/5hE6NCoobhyEu6...,Rema,👍Ja,NaN,commercial african pop. So smooth and well-pro...,28,juni,2023,NaN
4,https://www.roskilde-festival.dk/da/years/2023...,US,Lorna Shore,Avalon,Onsdag 28. juni 2023,https://www.instagram.com/lornashore/?hl=da,https://open.spotify.com/artist/6vXYoy8ouRVib3...,Armand Hammer,😌Tømmermændsmusik,NaN,chill jazz akkorder + rap,28,juni,2023,NaN
